In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa

In [ ]:
from VideoSwinTransformer import model_configs, I3DHead_tf , CosineAnnealingWithWarmupSchedule

In [ ]:
def get_model(num_classes, backbone ,shape_of_input=(3,32,224,224)):
    inputs = tf.keras.Input(shape_of_input[1:])
    
    x = backbone(inputs, training= True)
    outputs = I3DHead_tf(num_classes, 768, training=True)(x)
    return tf.keras.Model(inputs, outputs)

In [ ]:
# loading the tf2 model with converted pre-trained weights
path_to_model_weights = "/Users/mohammadshoaib/Codes/tensorflow-test/GSOC-22-Video-Swin-Transformers/swin_tiny_patch244_window877_kinetics400_1k_tf"
backbone = tf.keras.models.load_model(path_to_model_weights)

# uncomment the following line if you want to initialize the backbone model with random values
# backbone = SwinTransformer3D(**cfg, shape_of_input=shape_of_input)

In [ ]:
batch_size = 1
num_frames = 32
resolution = 224
channels = 3
sample_count = 10
num_classes = 5

shape_of_input = (batch_size, 3, num_frames, resolution,resolution)

In [ ]:
videos = [tf.random.normal(shape_of_input) for _ in sample_count]
labels = tf.random.uniform(shape=[sample_count], minval=0, maxval=num_classes, dtype='int64')

In [ ]:
num_epochs = 3

warmup_epoch = 2
total_steps = int(num_epochs * sample_count / batch_size)

# Compute the number of warmup batches.
warmup_steps = int(warmup_epoch * sample_count / batch_size)

train_steps = sample_count// batch_size
total_train_steps = train_steps * num_epochs
test_steps = sample_count // batch_size

In [ ]:
model = get_model(num_classes,  backbone,shape_of_input=shape_of_input)

## backbone model has a multiplier of 0.1 for learning rate. To implement that MultiOptimizer is being used
lr_backbone = CosineAnnealingWithWarmupSchedule(learning_rate_base=.001,
                                    total_steps=total_steps,
                                    warmup_learning_rate=0.0,
                                    warmup_steps=warmup_steps,
                                    hold_base_rate_steps=0)
lr_classifier = CosineAnnealingWithWarmupSchedule(learning_rate_base=.01,
                                        total_steps=total_steps,
                                        warmup_learning_rate=0.0,
                                        warmup_steps=warmup_steps,
                                        hold_base_rate_steps=0)

optimizer_backbone = tfa.optimizers.AdamW(weight_decay= 0.05,learning_rate=lr_backbone, beta_1= 0.9, beta_2=0.999, epsilon=1e-8)
optimizer_classifier = tfa.optimizers.AdamW(weight_decay= 0.05,learning_rate=lr_classifier, beta_1= 0.9, beta_2=0.999, epsilon=1e-8)

optimizers_and_layers = [(optimizer_backbone, model.layers[1]), (optimizer_classifier, model.layers[2])]
optimizer = tfa.optimizers.MultiOptimizer(optimizers_and_layers)

loss_obj = tf.keras.losses.CategoricalCrossentropy(from_logits=True,label_smoothing=0.1,
                                    reduction=tf.keras.losses.Reduction.SUM)

metrics=["top_k_categorical_accuracy"] 

model.compile(loss=loss_obj, optimizer=optimizer, metrics=metrics)

In [ ]:
results = model.fit(
    dist_train_dataset,
    validation_data=dist_test_dataset,
    epochs=num_epochs,
    steps_per_epoch=train_steps,
    validation_steps=test_steps,
    callbacks=[],
    validation_freq=1,
    verbose=1)